In [1]:
ls

Mehmet Uzel HW5.ipynb


In [3]:
import numpy as np

mean 0, std 1 olan inputlarla mean 0 std 1 olan weightleri carpinca olusan elemanların distribution inin mean ve std degerleri neler ve denen oyleler

In [33]:
x = np.random.normal(0,1,10000000)
w = np.random.normal(0,1,10000000)

In [34]:
print(np.mean(x))
print(np.std(x))

-0.00038973902249377714
0.9998680326434076


In [35]:
print(np.mean(w))
print(np.std(w))

-1.2000873770818549e-05
1.00012989401046


In [36]:
print(np.mean(w*x))
print(np.std(w*x))

-0.00013645843664645838
0.9998552366295783


standart sapma 1 ve ortalama 0 olduğunda elimizdeki değerler de genelde ya 1 ya da -1 olmaktadır. Bu değerleri çarptığımızda etkisiz eleman 1 ile çarpacağımızdan aslında değerler de farketmeyecek ve yine benzer ortalama ve standart sapma hesaplayacağız.

In [37]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

from PIL import Image
from pathlib import Path
from matplotlib import pyplot as plt

import os
import math

In [38]:
PATH= Path("data/mnist")

In [39]:
def log_softmax(x): 
    return (x.exp()/(x.exp().sum(-1,keepdim=True)) + 1e-20).log()

In [40]:
def nll(preds, actuals): 
    return -preds[range(actuals.shape[0]), actuals].mean()

In [41]:
def validation_acc(model):
    return torch.stack([accuracy(model(xb), yb) for xb, yb in valid_dl]).mean().item()

In [42]:
def accuracy(preds, yb): 
    return (torch.argmax(preds, dim=1, keepdim = True)==yb).float().mean()

In [43]:
def loss_func(preds, targets):
    preds = log_softmax(preds)
    return nll(preds, targets)

In [44]:
def train(model, epochs=5, valid_epoch=5):
    for epoch in range(epochs):
        for xb, yb in train_dl:
            
            preds = model(xb)
            loss = loss_func(preds, yb.squeeze())
            loss.backward()
            optim.step()
            optim.zero_grad()
        
        if epoch % valid_epoch == 0:
            print(validation_acc(model))

In [45]:
def _get_files(p, fs, extensions = None):
    p = Path(p) # to support / notation
    res = [p/f for f in fs if not f.startswith(".") 
           and ((not extensions) or f'.{f.split(".")[-1].lower()}' in extensions)]
    return res

In [46]:
class Dataset():
    def __init__(self, x, y): 
        self.x,self.y = x,y
    def __len__(self): 
        return len(self.x)
    def __getitem__(self, i): 
        return self.x[i].view(-1,1,28,28),self.y[i]


class DataLoader():
    def __init__(self, ds, bs): 
        self.ds, self.bs = ds, bs
    def __iter__(self):
        n = len(self.ds)
        l = torch.randperm(n)

        
        for i in range(0, n, self.bs): 
            idxs_l = l[i:i+self.bs]
            yield self.ds[idxs_l]

In [47]:
def create_ds_from_file(src):
    imgs, labels = [], []
    
    for label in range(10):
        path = src/str(label)
        print(path)
        t = [o.name for o in os.scandir(path)]
        t = _get_files(path, t, extensions = [".jpg", ".png"])
        for e in t:
            l = [np.array(Image.open(e)).reshape(28*28)]
            imgs += l
        labels += ([label] * len(t))
    return torch.tensor(imgs,  dtype=torch.float32), torch.tensor(labels, dtype=torch.long).view(-1,1)

In [48]:
train_x, train_y = create_ds_from_file(PATH/"train")

data/mnist/train/0
data/mnist/train/1
data/mnist/train/2
data/mnist/train/3
data/mnist/train/4
data/mnist/train/5
data/mnist/train/6
data/mnist/train/7
data/mnist/train/8
data/mnist/train/9


In [49]:
val_x,val_y = create_ds_from_file(PATH/"validation")

data/mnist/validation/0
data/mnist/validation/1
data/mnist/validation/2
data/mnist/validation/3
data/mnist/validation/4
data/mnist/validation/5
data/mnist/validation/6
data/mnist/validation/7
data/mnist/validation/8
data/mnist/validation/9


In [50]:
mean = train_x.mean()
std = train_x.std()

train_x=(train_x-mean)/std
mean, std, train_x.mean(), train_x.std()

(tensor(34.3446), tensor(77.5014), tensor(-6.8550e-05), tensor(1.0000))

In [51]:
val_x = (val_x-mean)/std
val_x.mean(), val_x.std()

(tensor(0.0042), tensor(1.0038))

In [52]:
train_ds = Dataset(train_x, train_y)
valid_ds = Dataset(val_x,val_y)

In [53]:
train_dl = DataLoader(train_ds, 256)
valid_dl = DataLoader(valid_ds, 256)

In [54]:
x, y = next(iter(train_dl))

In [55]:
x.shape

torch.Size([256, 1, 28, 28])

In [56]:
def stats(x):
    print(f"Mean: {x.mean()}, Std: {x.std()}")

In [57]:
def gain(a):
    return math.sqrt(2.0 / (1 + a**2))

In [58]:
def kaiming_uniform(x, a):
    n = x[0].shape.numel()
    std = gain(a) / math.sqrt(n)
    bound = math.sqrt(4.5) * std
    x.data.uniform_(-bound, bound)

In [59]:
class Func(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x): 
        return self.func(x)

In [60]:
def flatten(x):      
    return x.view(x.shape[0], -1)

In [66]:
model = nn.Sequential(
        nn.Conv2d(1, 64, 5, padding=2,stride=1), nn.ReLU(),
        nn.Conv2d(64, 16, 3, padding=1,stride=1), nn.ReLU(),
        nn.Conv2d(16, 32, 3, padding=1,stride=1), nn.ReLU(),
        nn.Conv2d(32, 32, 3, padding=2,stride=1), nn.ReLU(),
        nn.AdaptiveAvgPool2d(1),
        Func(flatten),
        nn.Linear(32,10)
)

In [67]:
for l in model:
    if isinstance(l, nn.Conv2d):
        kaiming_uniform(l.weight, a = 0)
        l.bias.data.zero_()

In [68]:
temp = model(x)
stats(temp)

Mean: 0.23014140129089355, Std: 0.9474882483482361


In [69]:
optim = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-3)

In [70]:
train(model,80,10)

0.23210392892360687
0.5001816749572754
0.8340297937393188
0.8870820999145508
0.9267805218696594
0.9367732405662537
0.9456758499145508
0.9526708126068115
